# Getting started with prompty


**Learning Objectives** - Upon completing this tutorial, you should be able to:

- Write LLM application using prompty and visualize the trace of your application.
- batch run prompty against multi lines of data.


## 0. Install dependent packages

In [1]:
%%capture --no-stderr
%pip install promptflow-core

## 1. Execute a Prompty

Prompty is a file with .prompty extension for developing prompt template. 
The prompty asset is a markdown file with a modified front matter. 
The front matter is in yaml format that contains a number of metadata fields which defines model configuration and expected inputs of the prompty.

In [17]:
with open("prompty/basic.prompty") as fin:
    print(fin.read())

---
name: Basic Prompt
description: A basic prompt that uses the chat API to answer questions
model:
    api: chat
    configuration:
        type: azure_openai
        azure_deployment: gpt-4-0125-Preview
    parameters:
        max_tokens: 128
        temperature: 0.2
inputs:
  question:
    type: string
sample:
  "question": "Who is the most famous person in the world?"
---
system:
You are an AI assistant who helps people find information.
As the assistant, you answer questions briefly, succinctly. 

user:
{{question}}


In [18]:
import os
from dotenv import load_dotenv

load_dotenv("credentials.env")

True

In [20]:
from promptflow.core import Prompty

# load prompty as a flow
f = Prompty.load(source="prompty/basic.prompty")

# execute the flow as function
result = f(question="What is the capital of France?")
result

'The capital of France is Paris.'

You can override configuration with `AzureOpenAIModelConfiguration` and `OpenAIModelConfiguration`.

In [22]:
from promptflow.core import AzureOpenAIModelConfiguration, OpenAIModelConfiguration

# override configuration with AzureOpenAIModelConfiguration
configuration = AzureOpenAIModelConfiguration(
    # azure_endpoint="${env:AZURE_OPENAI_ENDPOINT}",  # Use ${env:<ENV_NAME>} to surround the environment variable name.
    # api_key="${env:AZURE_OPENAI_API_KEY}",
    azure_deployment="gpt-4o",
)

# override configuration with OpenAIModelConfiguration
# configuration = OpenAIModelConfiguration(
#     base_url="${env:OPENAI_BASE_URL}",
#     api_key="${env:OPENAI_API_KEY}",
#     model="gpt-3.5-turbo"
# )

override_model = {"configuration": configuration, "parameters": {"max_tokens": 512}}

# load prompty as a flow
f = Prompty.load(source="prompty/basic.prompty", model=override_model)

# execute the flow as function
result = f(question="What is the capital of France?")
result

'The capital of France is Paris.'

### Visualize trace by using start_trace

In [23]:
from promptflow.tracing import start_trace

# start a trace session, and print a url for user to check trace
start_trace()

Re-run below cell will collect a trace in trace UI.

In [24]:
# rerun the function, which will be recorded in the trace
question = "What is the capital of Japan?"
ground_truth = "Tokyo"
result = f(question=question)
result

'Tokyo.'

### Eval the result 

Note: the eval flow returns a `json_object`.

In [25]:
# load prompty as a flow
eval_flow = Prompty.load("prompty/eval.prompty")
# execute the flow as function
result = eval_flow(question=question, ground_truth=ground_truth, answer=result)
result

{'score': '5', 'explanation': 'Tokyo is the capital of Japan.'}